In [32]:
import os
import sys
import subprocess
from contextlib import contextmanager
import signal
import filecmp
import difflib
import shutil

# Configurations
toolchain_path = "/usr/bin/riscv64-unknown-elf-"
compiler_path = "./build/SYSY-compiler"
test_folder="./testcases"
sylib_path = "./RISCVLib/sylib.c"
# xxx.sy 会被复制到./function_test/test.c
temp_path = "./function_test"
pass_args=[]
Tolerance = True

CE_list = [] # Compile Error
AE_list = [] # Assembler Error
LE_list = [] # Linker Error
Time_Out = []
WA_list = []
AC_list = []
TLE_list = []
BadTest_list = []

Prob_list = []
test_list = []

In [33]:
# grab all the testcases
# find files recursively
def GrabFile():
    for root, dirs, files in os.walk(test_folder):
        for file in files:
            if file.endswith(".sy"):
                test_list.append(os.path.join(root, file))

In [34]:
def CopyFileToTemp(test):
    # No Bad Test Now
    shutil.copy(test,temp_path+"/test.c")
    if os.path.exists(test[:-2]+"in"):
        shutil.copy(test[:-2]+"in",temp_path+"/test.in")
    if os.path.exists(test[:-2]+"out"):
        shutil.copy(test[:-2]+"out",temp_path+"/test.out")

In [35]:
def TryCompile(original_case_path):
    compile_args=[compiler_path, temp_path + "/test.c"]
    for arg in pass_args:
        compile_args.append(arg)
    try:
        ret = subprocess.run(compile_args,timeout=60)
    except subprocess.TimeoutExpired:
        Time_Out.append(original_case_path)
        print("Timeout: "+original_case_path)
        return False
    if ret.returncode != 0:
        CE_list.append(original_case_path)
        print("CE: "+original_case_path)
        return False
    return True

In [36]:
def TryAssemble(original_case_path):
    try:
        ret = subprocess.run([toolchain_path+"as", temp_path + "/test.s", "-o", temp_path + "/test.o"],timeout=10)
    except subprocess.TimeoutExpired:
        print("IMPOSSIBLE, AS TIMEOUT")
        exit(-1)
    if ret.returncode != 0:
        AE_list.append(original_case_path)
        print("AE: "+original_case_path)
        return False
    return True

In [37]:
def TryLink(orginal_case_path):
    try:
        ret = subprocess.run([toolchain_path+"gcc", temp_path + "/test.o", temp_path+"/sylib.o", "-o", temp_path + "/test"],timeout=10)
    except subprocess.TimeoutExpired:
        print("IMPOSSIBLE, LD TIMEOUT")
        exit(-1)
    if ret.returncode != 0:
        LE_list.append(orginal_case_path)
        print("LE: "+orginal_case_path)
        return False
    return True

In [38]:

def RunTest(original_case_path):
    try:
        if not os.path.exists(temp_path+"/test.in"):
            ret = subprocess.run(["qemu-system-riscv64", temp_path + "/test"],stdout=subprocess.PIPE,stderr=subprocess.PIPE,timeout=30)
        else:
            ret = subprocess.run(["qemu-system-riscv64", temp_path + "/test"],stdin=open(temp_path+"/test.in"),stdout=subprocess.PIPE,stderr=subprocess.PIPE,timeout=30)
    except subprocess.TimeoutExpired:
        print("TIMEOUT ERROR: "+original_case_path)
        TLE_list.append(original_case_path)
        return False
    
    if not os.path.exists(temp_path + "/test.out"):
        BadTest_list.append(original_case_path)
        return False
    else:
        out_file=temp_path + "/test.out"
    
    dump_str=ret.stdout.decode()
    # dump_str1 = ret1.stdout.decode()
    # remove whitesspace in the end
    # dump_str=dump_str.rstrip()
    # if dump_str1 and not dump_str1.endswith('\n'):
    #     dump_str1 += "\n"
    # if not dump_str1.endswith(''):
    #     print("RISCV Test Error")
    if dump_str and not dump_str.endswith('\n'):
        dump_str += "\n"
    dump_str += str(ret.returncode) + "\n"
    std_output=open(out_file).read()
    diff = difflib.unified_diff(dump_str.splitlines(), std_output.splitlines(), lineterm='')
    if(len(list(diff))!=0):
        print("Wrong Answer: "+original_case_path)
        WA_list.append(original_case_path)
        print(original_case_path)
        return False
    
    return True

In [39]:
def Tolerant(FuncList,original_file_name):
    for func in FuncList:
        ret = func(original_file_name)
        if ret == False and Tolerance == False:
            print("No Tolerence, exit")
            sys.exit(-1)
        if ret == False:
            return False
    return True

In [40]:
def TestSingleCase(test):
    CopyFileToTemp(test)
    if Tolerant([TryCompile,TryAssemble,TryLink, RunTest],test)==False:
        return False
    AC_list.append(test)
    return True

In [41]:
# Compile the sylib.c
subprocess.run([toolchain_path+"gcc", sylib_path, "-c", "-o", temp_path+"/sylib.o"])

./RISCVLib/sylib.c:1:9: fatal error: stdio.h: No such file or directory
    1 | #include<stdio.h>
      |         ^~~~~~~~~
compilation terminated.


CompletedProcess(args=['/usr/bin/riscv64-unknown-elf-gcc', './RISCVLib/sylib.c', '-c', '-o', './function_test/sylib.o'], returncode=1)

In [42]:
# if len(Prob_list) == 0:
#     GrabFile()
# else:
#     test_list=Prob_list
#     Prob_list.clear()
test_list=["./wa/18_prim.sy"]
    
for test in test_list:
    if test.endswith(".sy"):
        if TestSingleCase(test):
            print("Pass: "+test)

AE: ./wa/18_prim.sy


./function_test/test.s: Assembler messages:
./function_test/test.s:1: Error: junk at end of line, first unrecognized character is `-'
./function_test/test.s:2: Error: junk at end of line, first unrecognized character is `-'
./function_test/test.s:4: Error: junk at end of line, first unrecognized character is `%'
./function_test/test.s:5: Error: junk at end of line, first unrecognized character is `%'
./function_test/test.s:6: Error: junk at end of line, first unrecognized character is `%'
./function_test/test.s:7: Error: unrecognized opcode `a0%7%11%19'
./function_test/test.s:8: Error: junk at end of line, first unrecognized character is `%'
./function_test/test.s:9: Error: unrecognized opcode `a0%7%11%19'
./function_test/test.s:10: Error: unrecognized opcode `a0%7%11%19'
./function_test/test.s:11: Error: unrecognized opcode `a0%7%11%19'
./function_test/test.s:12: Error: junk at end of line, first unrecognized character is `-'
./function_test/test.s:13: Error: junk at end of line, firs

In [43]:
# print all wrong tests and mv one of them to temp_folder for debugging
print(len(AC_list))
print(len(WA_list))
print(len(CE_list))
print(len(AE_list))
print(len(LE_list))
print(len(Time_Out))
print(len(TLE_list))
print(len(BadTest_list))

0
0
0
1
0
0
0
0


In [44]:
def MV2WorkSpace(CasesList,k):
    case = CasesList[k]
    CopyFileToTemp(case)
    print(case)
    return
        
# MV2WorkSpace(WA_list,0)

print(WA_list)
print(CE_list)
print(Time_Out)
print(TLE_list)

[]
[]
[]
[]
